# Developing and testing the pryngles `cpixx` extension

In this notebook we want to test the `cpixx` extension using data collected from the application of the original `pixx` extension 

In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from pryngles import pixx
from pryngles import *
import ctypes
import glob
%load_ext wurlitzer

## Collecting data

You must first download file `reflection-test.pkl` from Google Drive and place it at `/tmp`

In [8]:
f=open("/tmp/reflection-test.pkl","rb")
save_values=pickle.load(f)
f.close()

Now let's read the values and:

- Create a file with the fourier coefficient matrices (`<object>-ntot_nx_ny_nz.mat`)
- Create a file with the gauss integrations points (`<object>-xmu.mat`)
- Accumulate the data for a given object in a pandas dataframe.

In [9]:
matrices=dict()
columns=["delazim","beta","eta","zeta","nmugs","nmat","nfou","af","S1","S2","S3","S4"]
for i,save in enumerate(save_values):
    args=save["args"]
    obj=save["obj"].replace(" ","_")
    """
    args are:
        0: Number of points (int)
        1: Delta azim (array Nx1)
        2: betas (array: Nx1)
        3: etas (array: Nx1)
        4: zetas
        5: nmugs
        6: nmat
        7: nfou
        8: xmu
        9: rfou
        10: af
    """
    
    #Read properties
    npoints=args[0]
    delta_azim=args[1]
    betas=args[2]
    etas=args[3]
    zetas=args[4]
    nmugs,nmat,nfou=args[5:8]
    xmu=args[8]
    rfou=args[9]
    nx,ny,nz=rfou.shape
    af=args[10]
    hash_mat=hash(rfou.sum())
    
    #Read stokes
    stokes=save["stokes"]
    
    #Save in dataframe
    data=pd.DataFrame(columns=columns)
    data["delazim"]=pd.Series(delta_azim)
    data["beta"]=pd.Series(betas)
    data["eta"]=pd.Series(etas)
    data["zeta"]=pd.Series(zetas)
    data["nmugs"]=nmugs
    data["nmat"]=nmat
    data["nfou"]=nfou
    data["af"]=pd.Series(af)
    data[["S1","S2","S3","S4"]]=pd.DataFrame(stokes)
    
    #Save matrix
    if not hash_mat in matrices.keys():    
        #Save matrix
        ntot=nx*ny*nz
        filename=f"{obj}-{ntot}_{nx}_{ny}_{nz}.mat"
        print(f"Saving file {filename}...")
        np.savetxt(filename,rfou.reshape(ntot,1))

        filename=f"{obj}-xmu-{len(xmu)}.mat"
        print(f"Saving file {filename}...")
        np.savetxt(filename,xmu)

        matrices[hash_mat]=dict(
            obj=obj,
            data=pd.DataFrame(columns=columns),
            rfou=rfou,
        )
    
    #Store data
    matrices[hash_mat]["data"]=pd.concat((matrices[hash_mat]["data"],data),ignore_index=True)
    #break
        
print("Done.")

Saving file planet-5292_63_21_4.mat...
Saving file planet-xmu-21.mat...
Saving file ring_forward-414000_60_20_345.mat...
Saving file ring_forward-xmu-20.mat...
Saving file ring_back-414000_60_20_345.mat...
Saving file ring_back-xmu-20.mat...
Done.


Save the interpolation matrices (the unique values) into interpolation files:

In [10]:
for hash_mat,item in matrices.items():
    obj=item["obj"]
    print(f"Saving interpolation matrix for {obj}")
    data=item["data"]
    print(f"Original number of interpolation points {len(data)}")
    data_clean=data.drop_duplicates()
    print(f"Number of interpolation points after removing duplicates {len(data_clean)}")
    filename=f"{obj}-interpolation.mat"
    print(f"Saving file {filename}...")
    np.savetxt(filename,data_clean)

Saving interpolation matrix for planet
Original number of interpolation points 696730
Number of interpolation points after removing duplicates 193758
Saving file planet-interpolation.mat...
Saving interpolation matrix for ring_forward
Original number of interpolation points 539136
Number of interpolation points after removing duplicates 231
Saving file ring_forward-interpolation.mat...
Saving interpolation matrix for ring_back
Original number of interpolation points 616535
Number of interpolation points after removing duplicates 320
Saving file ring_back-interpolation.mat...


## Check using pixx

Read data:

In [9]:
!ls *.mat

planet-5292_63_21_4.mat		ring_back-xmu-20.mat
planet-interpolation.mat	ring_forward-414000_60_20_345.mat
planet-xmu-21.mat		ring_forward-interpolation.mat
ring_back-414000_60_20_345.mat	ring_forward-xmu-20.mat
ring_back-interpolation.mat


Read data:

In [10]:
filename="planet-5292_63_21_4.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="planet-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="planet-xmu-21.mat"
xmu=np.loadtxt(filename)
#Compare:
#rfou_ori=matrices[401387150607385127]["rfou"]
#abs(rfou-rfou_ori).mean()

Check the first column:

In [11]:
interpolation[0]

array([1.04484516e+00, 3.06939428e+00, 4.99032903e-02, 2.50967097e-02,
       2.10000000e+01, 3.00000000e+00, 3.00000000e+00, 9.43232879e-05,
       4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01])

Read the input parameter values:

In [12]:
delazim,beta,eta,zeta,nmugs,nmat,nfou,af=interpolation[0,:8]
delazim,beta,eta,zeta,nmugs,nmat,nfou,af

(1.0448451569439827,
 3.069394277348945,
 0.04990329026929557,
 0.02509670973070432,
 21.0,
 3.0,
 3.0,
 9.432328787795567e-05)

And the stokes vector components:

In [32]:
stokes=interpolation[0,8:]
stokes

array([4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01])

Let's calculate the stokes vector using the pixx.reflection routine:

In [33]:
stokes_rec=pixx.reflection(
    1,#nvals
    [delazim],
    [beta],
    [eta],
    [zeta],
    int(nmugs),
    int(nmat),
    int(nfou),
    xmu,
    rfou,
    af
)
stokes_rec

array([[4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01]])

Which coincides with what is stored:

In [34]:
stokes-stokes_rec

array([[0., 0., 0., 0.]])

Let's do it for more stored interpolation values:

In [40]:
for i in range(10):
    delazim,beta,eta,zeta,nmugs,nmat,nfou,af=interpolation[i,:8]
    stokes_sto=interpolation[i,8:]
    stokes_rec=pixx.reflection(
        1,#nvals
        [delazim],
        [beta],
        [eta],
        [zeta],
        int(nmugs),
        int(nmat),
        int(nfou),
        xmu,
        rfou,
        af
    )
    print(stokes_sto-stokes_rec)

[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]
[[0. 0. 0. 0.]]


Let's do it for all:

In [49]:
delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

In [50]:
stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )

In [51]:
stokes_sto=interpolation[:,8:]

In [53]:
abs(stokes_sto-stokes_rec).sum()

0.0

It works!

The same thing can be repeated for ring

In [54]:
filename="planet-5292_63_21_4.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="planet-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="planet-xmu-21.mat"
xmu=np.loadtxt(filename)

delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )
stokes_sto=interpolation[:,8:]
abs(stokes_sto-stokes_rec).sum()

0.0

In [57]:
filename="ring_forward-414000_60_20_345.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="ring_forward-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="ring_forward-xmu-20.mat"
xmu=np.loadtxt(filename)

delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )
stokes_sto=interpolation[:,8:]
abs(stokes_sto-stokes_rec).sum()

0.0

In [60]:
filename="ring_back-414000_60_20_345.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="ring_back-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="ring_back-xmu-20.mat"
xmu=np.loadtxt(filename)

delazim=interpolation[:,0]
beta=interpolation[:,1]
eta=interpolation[:,2]
zeta=interpolation[:,3]

nmugs=int(interpolation[0,4])
nmat=int(interpolation[0,5])
nfou=int(interpolation[0,6])

af=interpolation[:,7]
nvals=len(af)

stokes_rec=pixx.reflection(
        nvals,
        delazim,
        beta,
        eta,
        zeta,
        nmugs,
        nmat,
        nfou,
        xmu,
        rfou,
        af
    )
stokes_sto=interpolation[:,8:]
abs(stokes_sto-stokes_rec).sum()

0.0

## How reshape works

In [71]:
matrix=np.array([[[1,2,3,4],[5,6,7,8],[9,10,11,12]],[[-1,-2,-3,-4],[-6,-6,-7,-8],[-9,-10,-11,-12]]])

In [75]:
nx,ny,nz=matrix.shape
ntot=nx*ny*nz
nx,ny,nz

(2, 3, 4)

In [76]:
matrix_reshap=matrix.reshape(ntot,1)
matrix_reshap

array([[  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ -1],
       [ -2],
       [ -3],
       [ -4],
       [ -6],
       [ -6],
       [ -7],
       [ -8],
       [ -9],
       [-10],
       [-11],
       [-12]])

In [80]:
matrix_recons=np.zeros((nx,ny,nz))
n=0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            matrix_recons[i][j][k]=matrix_reshap[n]
            n+=1

In [84]:
matrix_recons

array([[[  1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.],
        [  9.,  10.,  11.,  12.]],

       [[ -1.,  -2.,  -3.,  -4.],
        [ -6.,  -6.,  -7.,  -8.],
        [ -9., -10., -11., -12.]]])

## CPIXX using ctypes

In [2]:
libfile=glob.glob(Misc.get_data("../cpixx.*.so"))[0]
libfile

'/Users/jorgezuluagacallejas/Dropbox/MiInvestigacion/PapersEnProceso/Exorings/pryngles/src/pryngles/data/../cpixx.cpython-311-darwin.so'

In [3]:
#cpixx=ctypes.CDLL(Misc.get_data(library))
cpixx=ctypes.CDLL(libfile)

In [4]:
#Global variables
MAX_MAT=3
MAX_MUS=22
MAX_FOU=350
MAX_PIX=1000
def read_fourier(filename):
    """
    Read a file containing fourier coefficients produced by PyMieDAP

    Parameters:
    
        filename: string:
            File with the fourier coefficients.

    Returns:

        nmugs: int:
           Number of gaussian integration coefficients.

        nmat: int:
           Number of Fourier components.

        nfou: int:
           Number of Fourier coefficients.

        rfout: array (nmugs*nmat,nmugs,nfou):
           Matrix with the fourier coefficients for reflection.

        rtra: array (nmugs*nmat,nmugs,nfou): 
           Matrix with the fourier coefficients for transmission (if apply).

    """
    f=open(filename)

    #Read header
    nmat=0
    imu=0
    for i,line in enumerate(f):
        if '#' in line:
            continue
        data=line.split()
        if len(data)<3:
            if len(data)==1:
                if not nmat:
                    nmat=int(data[0])
                else:
                    nmugs=int(data[0])
                    xmu=np.zeros(nmugs)
            else:
                xmu[imu]=float(data[0])
                imu+=1
        else:
            break

    #Get core data
    data=np.loadtxt(filename,skiprows=i)
    nfou=int(data[:,0].max())+1

    rfou=np.zeros((MAX_MAT*MAX_MUS,MAX_MUS,MAX_FOU))
    rtra=np.zeros((MAX_MAT*MAX_MUS,MAX_MUS,MAX_FOU))

    #Read fourier coefficients
    for row in data:
        m,i,j=int(row[0]),int(row[1])-1,int(row[2])-1
        ibase=i*nmat
        rfou[ibase:ibase+3,j,m]=row[3:3+nmat]
        if len(row[3:])>nmat:
            rtra[ibase:ibase+3,j,m]=row[3+nmat:3+2*nmat]
            
    return nmat,nmugs,nfou,xmu,rfou,rtra

In [5]:
filename="fou_gasplanet.dat"
nmat,nmugs,nfou,xmu,rfou,rtra=read_fourier(filename)

In [6]:
nmat,nmugs,nfou

(3, 21, 3)

In [7]:
npix=1
phi=np.zeros(npix)
beta=np.zeros(npix)
theta0=np.zeros(npix)
theta=np.zeros(npix)
apix=np.zeros(npix)
Sarr=np.zeros((MAX_PIX,MAX_MAT+1))
shape=np.array([nmat,nmugs,nfou],dtype=int)

In [8]:
phi[0]=1.0448451569439827;
beta[0]=3.069394277348945;
theta0[0]=0.04990329026929557;
theta[0]=0.02509670973070432;
apix[0]=9.432328787795567e-05;

In [9]:
cpixx.reflection.argtypes = [
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=int),
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=float),
    np.ctypeslib.ndpointer(dtype=float),
]

In [10]:
cpixx.reflection(npix,phi,beta,theta0,theta,shape,xmu,rfou,apix,Sarr)

0

4.5978574249025603e-07 2.2512299728721978e-07 1.8344008005631274e-09 4.8964213134249535e-01 


In [11]:
Sarr[0]

array([4.59785742e-07, 2.25122997e-07, 1.83440080e-09, 4.89642131e-01])

Massive test:

In [12]:
data=np.loadtxt("planet-interpolation.mat")

In [13]:
npix=2
phi=data[:npix,0]
beta=data[:npix,1]
theta0=data[:npix,2]
theta=data[:npix,3]
apix=data[:npix,7]
Spixx=data[:npix,8:]

In [14]:
Spixx.shape

(2, 4)

In [15]:
npix=len(phi)
print(npix)

2


In [16]:
Sarr=np.zeros((MAX_PIX,MAX_MAT+1))

In [17]:
cpixx.reflection(npix,phi,beta,theta0,theta,shape,xmu,rfou,apix,Sarr)

0

4.5978574249025603e-07 2.2512299728721978e-07 1.8344008005631274e-09 4.8964213134249535e-01 
3.0764298300968855e+00 3.1156269695201195e+00 -5.3939660300185432e-01 1.0278063400529971e+00 


In [18]:
Sarr[1]

array([ 3.07642983,  3.11562697, -0.5393966 ,  1.02780634])

0.06713386589710574

In [22]:
np.random.rand(10,10)

array([[0.81487907, 0.24939891, 0.26122312, 0.52951497, 0.38251477,
        0.7349349 , 0.63143151, 0.30621508, 0.32178908, 0.09923947],
       [0.66509379, 0.65766778, 0.18032158, 0.89154235, 0.80931613,
        0.13943851, 0.86482976, 0.43150413, 0.10495243, 0.60990477],
       [0.13381602, 0.6564353 , 0.9151039 , 0.33814194, 0.99744972,
        0.35854345, 0.69953885, 0.32775088, 0.34920686, 0.90224016],
       [0.5978191 , 0.16936725, 0.40762404, 0.1817239 , 0.71449535,
        0.18688713, 0.56509655, 0.30894284, 0.36862986, 0.32941899],
       [0.9089598 , 0.31009955, 0.716284  , 0.71487116, 0.60366621,
        0.40821321, 0.79954942, 0.9233474 , 0.93608776, 0.6123023 ],
       [0.84788317, 0.72470247, 0.07010189, 0.44423283, 0.38355007,
        0.19705982, 0.24949129, 0.75799769, 0.25960549, 0.43143455],
       [0.86102735, 0.88586562, 0.46694527, 0.06156265, 0.48731098,
        0.5580955 , 0.63909957, 0.22759504, 0.39378637, 0.80723318],
       [0.00342398, 0.95340133, 0.1020700